## 5.3 Model Serialization

<div style="text-align: right"> Initial issue : 2025.05.07 </div>
<div style="text-align: right"> last update : 2025.05.07 </div>

직렬화란 모델을 저장 가능한 형식으로 변환하는 과정임
직렬화의 목적
   - 모델 재사용 (재훈련 없이)
   - 모델 배포 및 공유 용이
   - 계산 리소스 절약
직렬화의 장점
   - 빠른 모델 로딩
   - 버전 관리 가능
   - 다양한 환경에서 사용 가능

모델 직렬화는 AI 개발 및 배포 과정에서 중요한 단계로, 효율적인 모델 관리와 재사용을 가능하게 함
`is_lc_serializable` 클래스 메서드로 실행하여 LangChain 클래스가 직렬화 가능한지 확인가능

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# 프롬프트 템플릿을 사용하여 질문을 생성합니다.
prompt = PromptTemplate.from_template("{fruit}의 색상이 무엇입니까?")

클래스(class) 에 대하여 직렬화 가능 여부를 확인

In [3]:
print(f"ChatOpenAI: {ChatOpenAI.is_lc_serializable()}")

ChatOpenAI: True


In [4]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
print(f"ChatOpenAI: {llm.is_lc_serializable()}")

ChatOpenAI: True


In [5]:
# 체인을 생성
chain = prompt | llm
chain.is_lc_serializable()

True

### Chain 직렬화: dumps, dumpd
체인 직렬화는 직렬화 가능한 모든 객체를 딕셔너리 또는 JSON 문자열로 변환하는 과정을 의미
- `dumps`: 객체를 JSON 문자열로 직렬화
- `dumpd`: 객체를 딕셔너리로 직렬화

In [6]:
chain

PromptTemplate(input_variables=['fruit'], input_types={}, partial_variables={}, template='{fruit}의 색상이 무엇입니까?')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7f7e922b0450>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7f7e927a3210>, root_client=<openai.OpenAI object at 0x7f7e924678d0>, root_async_client=<openai.AsyncOpenAI object at 0x7f7e92275b50>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [7]:
from langchain_core.load import dumpd, dumps

dumpd 예시

In [8]:
dumpd_chain = dumpd(chain)
dumpd_chain

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'runnable', 'RunnableSequence'],
 'kwargs': {'first': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
   'kwargs': {'input_variables': ['fruit'],
    'template': '{fruit}의 색상이 무엇입니까?',
    'template_format': 'f-string'},
   'name': 'PromptTemplate'},
  'last': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'chat_models', 'openai', 'ChatOpenAI'],
   'kwargs': {'model_name': 'gpt-3.5-turbo',
    'temperature': 0.0,
    'openai_api_key': {'lc': 1, 'type': 'secret', 'id': ['OPENAI_API_KEY']}},
   'name': 'ChatOpenAI'}},
 'name': 'RunnableSequence'}

In [9]:
type(dumpd_chain)

dict

dumps 예시

In [10]:
dumps_chain = dumps(chain)
dumps_chain

'{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "runnable", "RunnableSequence"], "kwargs": {"first": {"lc": 1, "type": "constructor", "id": ["langchain", "prompts", "prompt", "PromptTemplate"], "kwargs": {"input_variables": ["fruit"], "template": "{fruit}\\uc758 \\uc0c9\\uc0c1\\uc774 \\ubb34\\uc5c7\\uc785\\ub2c8\\uae4c?", "template_format": "f-string"}, "name": "PromptTemplate"}, "last": {"lc": 1, "type": "constructor", "id": ["langchain", "chat_models", "openai", "ChatOpenAI"], "kwargs": {"model_name": "gpt-3.5-turbo", "temperature": 0.0, "openai_api_key": {"lc": 1, "type": "secret", "id": ["OPENAI_API_KEY"]}}, "name": "ChatOpenAI"}}, "name": "RunnableSequence"}'

In [11]:
type(dumps_chain)

str

### Pickle 파일로 저장하기
Pickle 파일은 Python 객체를 바이너리 형태로 직렬화하는 포맷임
Pickle 파일 특징  
   - Python 전용 (다른 언어와 호환 불가)
   - 대부분의 Python 데이터 타입 지원 (리스트, 딕셔너리, 클래스 등)
   - 객체의 상태와 구조를 그대로 보존

Pickle 파일 장점  
   - 효율적인 저장 및 전송
   - 복잡한 객체 구조 유지
   - 빠른 직렬화/역직렬화 속도

Pickle 파일 단점  
   - 보안 위험 (신뢰할 수 없는 데이터 역직렬화 시 주의 필요)
   - 사람이 읽을 수 없는 바이너리 형식

- `pickle.dump()`: 객체를 파일에 저장
- `pickle.load()`: 파일에서 객체 로드

dumpd_chain을 pickle 파일로 저장

In [12]:
import pickle

# fuit_chain.pkl 파일로 직렬화된 체인을 저장합니다.
with open("fruit_chain.pkl", "wb") as f:
    pickle.dump(dumpd_chain, f)

json 파일 형식으로 저장

In [13]:
import json

with open("fruit_chain.json", "w") as fp:
    json.dump(dumpd_chain, fp)

### 저장한 모델 불러오기: load

In [14]:
import pickle

# pickle 파일을 로드합니다.
with open("fruit_chain.pkl", "rb") as f:
    loaded_chain = pickle.load(f)

In [15]:
loaded_chain

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'runnable', 'RunnableSequence'],
 'kwargs': {'first': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
   'kwargs': {'input_variables': ['fruit'],
    'template': '{fruit}의 색상이 무엇입니까?',
    'template_format': 'f-string'},
   'name': 'PromptTemplate'},
  'last': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'chat_models', 'openai', 'ChatOpenAI'],
   'kwargs': {'model_name': 'gpt-3.5-turbo',
    'temperature': 0.0,
    'openai_api_key': {'lc': 1, 'type': 'secret', 'id': ['OPENAI_API_KEY']}},
   'name': 'ChatOpenAI'}},
 'name': 'RunnableSequence'}

이 것을 langchain load 함수로 읽어서 사용하기

In [16]:
from langchain_core.load import load

# 체인을 로드합니다.
chain_from_file = load(loaded_chain)

# 체인을 실행합니다.
print(chain_from_file.invoke({"fruit": "사과"}))

/tmp/ipykernel_97354/741098595.py:4: LangChainBetaWarning: The function `load` is in beta. It is actively being worked on, so the API may change.
  chain_from_file = load(loaded_chain)


content='사과의 색상은 주로 빨간색이지만, 녹색, 노란색, 주황색 등 다양한 색상의 사과도 있습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 24, 'total_tokens': 76, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BUY2lZD7bS6ZYWOEIKx5PTzpBDjdm', 'finish_reason': 'stop', 'logprobs': None} id='run-eb0e49b8-6504-47fc-b5ac-f0e71f403eea-0' usage_metadata={'input_tokens': 24, 'output_tokens': 52, 'total_tokens': 76, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [17]:
from langchain_core.load import load, loads

load_chain = load(
    loaded_chain, secrets_map={"OPENAI_API_KEY": os.environ["OPENAI_API_KEY"]}
)

# 불러온 체인이 정상 동작하는지 확인합니다.
load_chain.invoke({"fruit": "사과"})

AIMessage(content='사과의 색상은 주로 빨간색이지만, 녹색, 노란색, 주황색 등 다양한 색상의 사과도 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 24, 'total_tokens': 76, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BUY2mrtg9GxDJZRKcqlJQ88KcS3UD', 'finish_reason': 'stop', 'logprobs': None}, id='run-7a19e4ae-969c-4a47-9e4d-b968eb759317-0', usage_metadata={'input_tokens': 24, 'output_tokens': 52, 'total_tokens': 76, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [18]:
with open("fruit_chain.json", "r") as fp:
    loaded_from_json_chain = json.load(fp)
    loads_chain = load(loaded_from_json_chain)

In [19]:
loads_chain.invoke({"fruit": "사과"})

AIMessage(content='사과의 색상은 주로 빨간색이지만, 녹색, 노란색, 주황색 등 다양한 색상의 사과도 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 24, 'total_tokens': 76, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BUY2oayOHLb3LojkDaoGk4JoXVV1i', 'finish_reason': 'stop', 'logprobs': None}, id='run-bf75ccb5-3715-4df2-b010-6430aa1229e4-0', usage_metadata={'input_tokens': 24, 'output_tokens': 52, 'total_tokens': 76, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})